In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 9
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
# Class 1
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
delay.set_service(jobclass1, Exp(0.5))

# Class 2
queue1.set_service(jobclass2, Exp(1.0))
queue2.set_service(jobclass2, Exp(2.0))
delay.set_service(jobclass2, Exp(0.2))

In [5]:
# Set routing matrix
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/2052350308563438810/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.092072s.
  Station JobClass    QLen    Util    RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.0786  1.0786   1.9821  1.9821  0.5433  0.5434
1  Delay1   class2  2.2985  2.2985   5.0546  5.0546  0.4624  0.4626
2  Queue1   class1  8.9214  0.5414  16.4290  8.2145  0.5434  0.5433
3  Queue1   class2  7.7015  0.4553  16.6717  8.3358  0.4626  0.4624
4  Queue2   class1  0.5708  0.2640   1.0145  0.5073  0.5434  0.5434
5  Queue2   class2  0.4647  0.2335   1.0317  0.5158  0.4626  0.4626
6    Join   class1  8.4049  0.0000   7.7213  7.7213  1.0828  0.5433
7    Join   class2  7.2518  0.0000   7.7680  7.7680  0.9268  0.4624
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.161029s.
  Station JobClass    QLen    Util    RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.0241  1.0241   2.0000  2.0000  0.512